# CFD Examples #

In [1]:
import numpy as np
import matpltotlib.pyplot as plt

ModuleNotFoundError: No module named 'matpltotlib'

## 2D Steady State Heat Transfer in Square Plate

Governing Equation: ∂²T/∂x² + ∂²T/∂y² = ∇²T = 0

Boundary Condition: T(x=0) = 100, T(y=0) = T(y=100) = T(x=100) = 0

Discretized: T(i,j) = ( T(i-1,j-1) + T(i+1,j-1) + T(i-1,j+1) + T(i+1,j+1) ) / 4
=> average of 4 neighbouring points

Can use a simple 5-point 2D Stencil Method due to local PDE discretization

In [ ]:
# grid size
x_size = 100
y_size = 100

# Note: index 0 and -1 are outside edges 
T = np.zeros((y_size+2, x_size+2))
T[1:x_size+1, 0] = 100

# define plots
fig, ax = plt.subplots(4, 2, sharex=True, figsize=(5, 8))

# iteration to convergence, with intermediate plotting every 800 iterations
for i in range(8):

    for j in range(500):
        # Tn stores values from previous iteration, used to obtain new T
        Tn = T.copy()   

        # 5-point stencil method (2D) - only for special cases of local PDE discretizations
        T[1:-1, 1:-1] = (Tn[0:-2, 1:-1] + Tn[2:, 1:-1] + Tn[1:-1, 0:-2] + Tn[1:-1, 2:]) / 4

        T[0, :] = 0
        T[-1, :] = 0
        T[:, 0] = 100
        T[:, -1] = 0
        
    axi = ax[i%4][i//4]
    axi.set_title(f"{(i+1)*500} iterations")
    axi.pcolormesh(range(x_size+2), range(y_size+2), T, cmap="jet")

fig.tight_layout()  # ensures title and subplots are properly spaced with no overlaps
plt.show() # shows progressively converging temperature distribution in the plate

